In [1]:
import csv
import ast
import numpy as np
import time
import random

start = time.time()

# --- Load BB (Billboard data) ---
BB = []
with open("BB_NYC.csv", 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        BB.append({'B_id': int(row['B_index'])})

# --- Load ub (User influence data) ---
ub = []
with open("new_ub.csv", 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        row['Influenced Billboards'] = ast.literal_eval(row['Influenced Billboards'])
        ub.append(row)

# --- Load Dem (Demand values) ---
Dem = []
with open("advertiser_1.csv", 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        Dem.append(float(row['Demand']))

# BillboardSlot class
class BillboardSlot:
    def __init__(self, idx):
        self.idx = idx

    def __repr__(self):
        return f"Slot({self.idx})"

# Convert BB to billboard slots
BS = [BillboardSlot(item['B_id']) for item in BB[:5000]]

In [2]:
# Influence function
def influence(S):
    influence_score = 0.0
    slot_ids = {s.idx if isinstance(s, BillboardSlot) else s for s in S}
   
    for row in ub:
        influenced_billboards = row.get('Influenced Billboards', {})
       
        intersection_result = {int(key): value for key, value in influenced_billboards.items() if int(key) in slot_ids}
       
        product = 1.0
        for value in intersection_result.values():
            product *= (1 - value)
        influence_score += (1 - product)
    return influence_score

In [3]:
import random

def random_pick(BS, Dem):
    available_slots = {slot.idx for slot in BS}  # All billboard IDs
    used_slots = set()                           # Keep track of globally used slots
    Si_list = [set() for _ in Dem]               # Slot sets per product

    for j in range(len(Dem)):
        demand = Dem[j]
        current_influence = 0.0

        while current_influence < demand:
            remaining = list(available_slots - used_slots - Si_list[j])
            if not remaining:
                print(f"No more slots available to fulfill demand for product {j}.")
                break

            selected = random.choice(remaining)
            Si_list[j].add(selected)
            current_influence = influence(Si_list[j])
            used_slots.add(selected)  # Ensure no reuse across products

        # Optional: notify if demand not met
        if current_influence < demand:
            print(f"Warning: Demand for product {j} not fully satisfied. Got {current_influence:.2f}, needed {demand:.2f}")

    return Si_list


In [4]:
# Run the allocation
result = random_pick(BS, Dem)
total_influence = 0
# Total unique slots
all_selected = set().union(*result)
# Print per-product slot selections
for i, slots in enumerate(result):
    print(f"Product {i+1}: Selected Slots = {sorted(list(slots))}, Influence = {influence(slots):.2f}")
    total_influence += influence(slots)
# Calculate total unique slots and total influence
print("\nTotal Selected Slots (unique):", len(all_selected))
print("Total Combined Influence:", total_influence)
print("Execution time:", round(time.time() - start, 2), "seconds")


Product 1: Selected Slots = [176, 417, 422, 669, 807, 926, 1118, 1158, 1235, 1328, 1540, 1549, 1558, 1847, 2061, 2186, 2346, 2450, 2477, 2835, 2884, 2995, 3157, 3515, 3554, 3938, 4113, 4291, 4343, 4364, 4413, 4418, 4520, 4736, 4773, 4864, 4929, 4949], Influence = 6.64
Product 2: Selected Slots = [169, 194, 280, 415, 456, 485, 518, 555, 661, 680, 763, 767, 796, 839, 968, 988, 1022, 1094, 1259, 1281, 1398, 1472, 1475, 1605, 1626, 1687, 1806, 2025, 2027, 2054, 2103, 2133, 2287, 2303, 2502, 2536, 2568, 2583, 2608, 2614, 2681, 2686, 2747, 2757, 2891, 3061, 3064, 3191, 3279, 3337, 3512, 3746, 3855, 3916, 4260, 4262, 4334, 4438, 4453, 4471, 4479, 4485, 4515, 4517, 4719, 4741], Influence = 4.06
Product 3: Selected Slots = [21, 821, 941, 1679, 1955, 2023, 2337, 2667, 2742, 2777, 3196, 3323, 3433, 3595, 3666, 3712, 3969, 4189, 4553, 4577, 4754, 4993], Influence = 4.82
Product 4: Selected Slots = [213, 241, 789, 1159, 1239, 1442, 1511, 1537, 1616, 1620, 1901, 1984, 2028, 2174, 2195, 2223, 2270, 2

In [5]:
h = len(Dem)
count = 0

for j in range(h):
    if influence(result[j]) >= Dem[j]:
        count += 1

print("Number of products whose demands are satisfied:", count, "out of", h)


Number of products whose demands are satisfied: 100 out of 100
